<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Evolutionary_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define the SimpleEvolution class
class SimpleEvolution:
    def __init__(self, model, population_size, mutation_rate):
        self.model = model
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.population = [self.mutate_model(self.create_model()) for _ in range(self.population_size)]

    def create_model(self):
        # Create a new instance of the model
        new_model = SimpleModel()
        new_model.load_state_dict(self.model.state_dict())  # Copy parameters from the base model
        return new_model

    def mutate_model(self, model):
        with torch.no_grad():
            for param in model.parameters():
                mutation = torch.randn_like(param) * self.mutation_rate
                param.add_(mutation)
        return model

    def select_best(self, fitness_scores):
        best_index = fitness_scores.argmax()
        return self.population[best_index]

    def evolve(self, fitness_function):
        fitness_scores = torch.tensor([fitness_function(model) for model in self.population])
        best_model = self.select_best(fitness_scores)
        self.population = [self.mutate_model(self.create_model()) for _ in range(self.population_size)]
        return best_model

# Define a simple model class
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Define a dummy fitness function
def fitness_function(model):
    # Dummy fitness function: just the negative sum of parameters
    with torch.no_grad():
        return -sum(p.sum() for p in model.parameters())

# Example usage
base_model = SimpleModel()
evolution = SimpleEvolution(base_model, population_size=10, mutation_rate=0.01)
best_model = evolution.evolve(fitness_function)

# Print the best model found
print(best_model)